In [ ]:
# import the necessary libraries
from google.colab import drive
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import spacy
from collections import Counter
import torch
import torchtext
import torch.nn as nn
import torch.optim as optim
import random

In [ ]:
# To run this code section, one needs to run it once to install torchtext, then restart the runtime and run again to import the necessary libraries
!pip install torchtext==0.6.0 --quiet
from torchtext.data import Field, BucketIterator

In [ ]:
drive.mount('/gdrive') # Only needed if your data is on google drive

Mounted at /gdrive


In [ ]:
# Read JSON data from file
with open('/gdrive/MyDrive/train_data1.json', 'r') as file:
    data = json.load(file)

In [ ]:
# Read JSON data from file
with open('/gdrive/MyDrive/test_data1_final.json', 'r') as file:
    test_data = json.load(file)

In [ ]:
import string

# Define the remove_punctuations function
def remove_punctuations(sentence):
    punctuations = list(string.punctuation)
    punctuations.append('।')
    punctuations.append('৷')
    punctuations.append('’')
    punctuations.append('‘')
    ans = ""
    for letter in sentence:
        if letter not in punctuations:
            ans += letter
    return ans

In [ ]:
import re

# Define a function to check if a sentence contains English words
def contains_english_words(sentence):
    return bool(re.search(r'[a-zA-Z]', sentence))

In [ ]:
# Process JSON data
source_sentences_train = []
target_sentences_train = []
len_hindi = []
len_english = []

id_train = []

for language_pair, language_data in data.items():
    if(language_pair == "English-Hindi"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"].lower()
              target = entry_data["target"].lower()
              source = remove_punctuations(source)
              target = remove_punctuations(target)
              if not contains_english_words(target):
                source_sentences_train.append(source)
                target_sentences_train.append(target)
                id_train.append(entry_id)
                len_hindi.append(len(source.split(' ')))
                len_english.append(len(target.split(' ')))

Language Pair: English-Hindi
  Data Type: Train


In [ ]:
# Calculate the maximum length of Hindi and English sentences
max_length_hindi = max(len_hindi)
max_length_english = max(len_english)

print(f"Maximum Length of Hindi Sentence: {max_length_hindi} words")
print(f"Maximum Length of English Sentence: {max_length_english} words")

Maximum Length of Hindi Sentence: 178 words
Maximum Length of English Sentence: 216 words


In [ ]:
len(target_sentences_train)

74604

In [ ]:
test_source = []
valid_id = []
len_val=[]


for language_pair, language_data in test_data.items():
    if(language_pair == "English-Hindi"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"].lower()
              source = remove_punctuations(source)
              test_source.append(source)
              valid_id.append(entry_id)
              len_val.append(len(source.split(' ')))

Language Pair: English-Hindi
  Data Type: Test


In [ ]:
test_source[1]

'what is the address for the event scheduled on the first of january'

In [ ]:
print(max(len_val))

155


In [ ]:
nlp_en = spacy.load('en_core_web_sm')
# Tokenize English sentences
tokenized_source_sentences = []
for sentence in source_sentences_train:
    doc = nlp_en(sentence)
    tokens = [token.text for token in doc]
    tokenized_source_sentences.append(tokens)

In [ ]:
tokenized_test_source = []
for sentence in test_source:
    doc = nlp_en(sentence)
    tokens = [token.text for token in doc]
    tokenized_test_source.append(tokens)

In [ ]:
pip install indic-nlp-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 17.1 MB/s eta 0:00:00


In [ ]:
from indicnlp.tokenize import indic_tokenize

In [ ]:
# Tokenize Hindi sentences
tokenized_target_sentences = []
for sentence in target_sentences_train:
    tokens = indic_tokenize.trivial_tokenize(sentence)
    tokenized_target_sentences.append(tokens)

In [ ]:
from torchtext.data import Dataset, Example, Field

def tokenize_english(text):
    return text

def tokenize_hindi(text):
    return text

class CustomTranslationDataset(Dataset):
    def __init__(self, source_sentences, target_sentences, source_field, target_field):
        fields = [('source', source_field), ('target', target_field)]
        examples = []
        for src, tgt in zip(source_sentences, target_sentences):
            examples.append(Example.fromlist([src, tgt], fields))
        super().__init__(examples, fields)

# Create Fields for English (source) and Hindi (target) text
english = Field(tokenize=tokenize_english,
                lower=True,
                init_token="<sos>",
                eos_token="<eos>")

hindi = Field(tokenize=tokenize_hindi,
              lower=True,
              init_token="<sos>",
              eos_token="<eos>")


train_dataset = CustomTranslationDataset(tokenized_source_sentences, tokenized_target_sentences, english, hindi)


# Build source and target vocabulary
english.build_vocab(train_dataset,max_size=30000, min_freq=3)
hindi.build_vocab(train_dataset,max_size=30000, min_freq=3)

In [ ]:
print(len(english.vocab), len(hindi.vocab))

20912 22534


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32

train_iterator = BucketIterator(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    sort_within_batch=True,
    sort_key=lambda x: len(x.source),
    device=device
)

In [ ]:
a = list(english.vocab.__dict__.values())
word_2_idx = dict(a[3])
idx_2_word = {}
for i,j in word_2_idx.items():
  idx_2_word[j] = i

In [ ]:
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderLSTM, self).__init__()
    self.hidden_size = hidden_size

    self.num_layers = num_layers

    self.dropout = nn.Dropout(p)
    self.tag = True

    self.embedding = nn.Embedding(input_size, embedding_size)

    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

  def forward(self, x):
    embedding = self.dropout(self.embedding(x))
    outputs, (hidden_state, cell_state) = self.LSTM(embedding)
    return hidden_state, cell_state

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size_encoder = len(english.vocab)
encoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
encoder_dropout = 0.5
encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size, hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)

EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(20912, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
)


In [ ]:
class DecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
    super(DecoderLSTM, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers

    self.output_size = output_size

    self.dropout = nn.Dropout(p)

    self.embedding = nn.Embedding(input_size, embedding_size)
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)
    self.fc = nn.Linear(hidden_size, output_size)


  def forward(self, x, hidden_state, cell_state):
    x = x.unsqueeze(0)
    embedding = self.dropout(self.embedding(x))
    outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))
    predictions = self.fc(outputs)
    predictions = predictions.squeeze(0)
    return predictions, hidden_state, cell_state

In [ ]:
input_size_decoder = len(hindi.vocab)
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
decoder_dropout = 0.5
output_size = len(hindi.vocab)

decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size, hidden_size, num_layers, decoder_dropout, output_size).to(device)
print(decoder_lstm)

DecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(22534, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=1024, out_features=22534, bias=True)
)


In [ ]:
for batch in train_iterator:
  print(batch.source.shape)
  print(batch.target.shape)
  break

x = batch.target[1]
print(x)

torch.Size([11, 32])
torch.Size([16, 32])
tensor([  654,     0,   129,    16,   349,    21,  4799,  1645,    16,   144,
           16,    87,    37,  1750,    16,   190,   154,    38, 20109,  6232,
         1227,   126, 14953,   190,    91,   106,  4575,  2141,   572,    23,
          937,  6406], device='cuda:0')


In [ ]:
class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM

  def forward(self, source, target, tfr=0.5):
    batch_size = source.shape[1]
    target_len = target.shape[0]
    target_vocab_size = len(hindi.vocab)
    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
    hidden_state, cell_state = self.Encoder_LSTM(source)
    x = target[0]
    for i in range(1, target_len):
      output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
      outputs[i] = output
      best_guess = output.argmax(1)
      x = target[i] if random.random() < tfr else best_guess
    return outputs

In [ ]:
# Hyperparameters
learning_rate = 0.001
step = 0

model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = hindi.vocab.stoi[""]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [ ]:
model

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(20912, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  )
  (Decoder_LSTM): DecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(22534, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=22534, bias=True)
  )
)

In [ ]:
def pred_trans(model, tokens, english, hindi, device, max_length=216):
    tokens.insert(0, english.init_token)
    tokens.append(english.eos_token)
    text_to_indices = [english.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)

    outputs = [hindi.vocab.stoi[""]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)
        if output.argmax(1).item() == hindi.vocab.stoi[""]:
            break

    translated_sentence = [hindi.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:]

In [ ]:
import time
from tqdm import tqdm
epoch_loss = 0.0
num_epochs = 10
sentence1 = "avoid alcohol and illicit drugs"

for epoch in range(num_epochs):
  start_time = time.time()
  print("Epoch - {} / {}".format(epoch+1, num_epochs))
  model.eval()
  doc = nlp_en(sentence1)
  tok = [token.text for token in doc]
  translated_sentence1 = pred_trans(model, tok, english, hindi,device, max_length=128)
  print(f"Translated example sentence 1: \n {translated_sentence1}")

  model.train(True)
  for batch_idx, batch in tqdm(enumerate(train_iterator), total=len(train_iterator)):
    input = batch.source.to(device)
    target = batch.target.to(device)
    output = model(input, target)
    output = output[1:].reshape(-1, output.shape[2])
    target = target[1:].reshape(-1)
    optimizer.zero_grad()
    loss = criterion(output, target)
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
    optimizer.step()
    step += 1
    epoch_loss += loss.item()

  # Calculate the time taken for the epoch
  end_time = time.time()
  epoch_time = end_time - start_time
  print(f"Time taken for epoch {epoch + 1}: {epoch_time:.2f} seconds")
  print("Epoch_Loss - {}".format(loss.item()))
  print()

print(epoch_loss / len(train_iterator))

Epoch - 1 / 10
Translated example sentence 1: 
 ['रासलीला', 'स्टैण्ड', 'स्टैण्ड', 'सोचते', 'खीरा', 'खीरा', 'बीजाणु', 'बीजाणु', 'संसदीय', 'सालाना', 'पहनने', 'कसरत', 'कसरत', 'भविष्य', 'कसरत', 'राजहंस', 'पकड़ने', 'पकड़ने', 'चूरमा', 'घोषित', '1817', 'झुंकू', 'पकड़ने', 'झुंकू', 'पूर्णांक', 'पास्कल', 'बाज़ार', 'पकड़ने', 'राजहंस', 'पकड़ने', 'माइक्रोडाइलूशन', 'चूरमा', 'झुंकू', 'झुंकू', 'तनु', 'तनु', 'गरमाहट', 'मृत', 'मृत', 'चूरमा', 'बेसन', 'सदाबहार', 'सदाबहार', 'हिमनदों', 'गंगा', 'नवाँ', 'अबू', 'अबू', 'बढ़त', 'बच्चो', '100000', 'स्टैण्ड', 'गरमाहट', 'साहेब', 'बंजी', 'साउंडट्रैक', 'सदाबहार', 'सदाबहार', 'सदाबहार', '1772', 'फॉर्म', 'शायद', 'पड़ती', 'परिधान', 'दोहराएँ', '146', '146', 'कमोडिटी', '1836', 'ऑन', 'ऑन', 'जंतु', 'आवेग', 'तब', 'पकड़ने', 'ज्यों', 'त्रिफला', 'आलाप', '1767', 'होतीं', 'शिकोह', 'वार्ता', 'श्रंगार', '1817', 'अथान', 'अनमोल', 'अनमोल', 'दूषित', 'अनमोल', 'अनमोल', 'संपादित', 'सेटिंग', 'मुक्केबाज़ी', 'मुक्केबाज़ी', 'भयावह', 'साहेब', 'विकिरण', 'झनझनाहट', 'जीतकर', 'सेठ', 'दबंग', 'प्रिक्स', 'पकड़न

100%|██████████| 2332/2332 [11:19<00:00,  3.43it/s]


Time taken for epoch 1: 682.11 seconds
Epoch_Loss - 4.160567283630371

Epoch - 2 / 10
Translated example sentence 1: 
 ['और', 'और', 'और', 'और', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '

100%|██████████| 2332/2332 [11:20<00:00,  3.43it/s]


Time taken for epoch 2: 680.66 seconds
Epoch_Loss - 3.0918641090393066

Epoch - 3 / 10
Translated example sentence 1: 
 ['और', 'और', 'और', 'और', 'और', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<p

...

Time taken for epoch 8: 682.49 seconds
Epoch_Loss - 2.5149829387664795

Epoch - 9 / 10
Translated example sentence 1: 
 ['प्रॉपर', 'और', 'और', 'डाइट', 'लें', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad

100%|██████████| 2332/2332 [11:20<00:00,  3.43it/s]


Time taken for epoch 9: 680.94 seconds
Epoch_Loss - 3.1478395462036133

Epoch - 10 / 10
Translated example sentence 1: 
 ['स्वस्थ', 'और', 'और', 'और', 'फैट', '<eos>', 'बचें', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'

100%|██████████| 2332/2332 [11:19<00:00,  3.43it/s]

Time taken for epoch 10: 680.47 seconds
Epoch_Loss - 3.0766711235046387

31.902432937283663


In [ ]:
hindi_pred = []

# Loop through the test source sentences and translate them
for tokens in tokenized_test_source:
  translation = pred_trans(model, tokens, english, hindi, device, max_length=216)
  cleaned_tokens = [token for token in translation if token not in ['<eos>', '<pad>']]
  pred = ' '.join(cleaned_tokens)
  print(pred)
  hindi_pred.append(pred)

Streaming output truncated to the last 5000 lines.
यह एक के लिए के रूप में उपयोग किया जाता है क्योंकि वे एक के के के लिए उपयोग करने के लिए उपयोग किया जाता है
उन्होंने करिश्मा सर्विसेज के और और और और और और भी भी शामिल हैं
इन सभी के में में में में में में से से से से से हो जाती है
चावल के साथ में सूखा औसत वर्षा औसत है है है
ऐनरोबिक टेक्स्ट नपुंसकता हस्तक्षेपों के के के के के के मंदक का उपयोग करते हैं
इसके अलावा देश के भी देशभर के भी देशभर के भी विश्‍व के भी भी
राजनीतिक के के लिए पहली बार के लिए के लिए के लिए में में ६८ का भी भी है
मुझे कुछ टॉक बैक के दें
एलेक्सा रोहन अभी एक ईमेल भेजा है
सरकार के स्टॉल के के के के के के के के के के के के के के के के के के के के में भी है
दो दो दो या दो या दो या दो या या दो या दो या दो साल से दो साल के बीच में
सिरोसिस की एक में की कमी बढ़ सकती है और हृदय की कमी से कारण में में से अधिक बढ़ सकती है
रोग के के बाद के बाद को भी करता है और बाद में बाद में बाद में बाद में है
केरल में एसा वाशिंगटन के के में में के के के में से भी
इसके साथ ही महत्वपूर्ण है है
वहां

In [ ]:
print(len(hindi_pred))
print(len(valid_id))

In [ ]:
import csv

# Create a list of dictionaries where each dictionary represents a row
data = [{'valid_id': valid_id[i], 'hindi_pred': hindi_pred[i]} for i in range(len(valid_id))]

# Specify the CSV file path
csv_file_path = '/gdrive/MyDrive/hindi3.csv'

# Define the column names
fields = ['valid_id', 'hindi_pred']

# Write the data to the CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    writer.writerows(data)

print(f'Saved predictions to {csv_file_path}')

NameError: ignored